# Two Qubit CNOT Gate with Piccolo.jl

In [1]:
using Revise
using CairoMakie
using LinearAlgebra
using Piccolo

## System Hamiltonian

The Hamiltonian for this two qubit system example is given by

$$
H(u(t)) = g \hat a^\dagger \hat a \hat b^\dagger \hat b + a_1(t) (\hat a + \hat a^\dagger) + i a_2(t) (\hat a - \hat a^\dagger) + a_3(t) (\hat b + \hat b^\dagger) + i a_4(t)(\hat b - \hat b^\dagger) 
$$

where $\hat a$ and $\hat b$ are the annihilation operators for the first and second qubit respectively, $g$ is the coupling strength, and $u_i(t)$ are the control functions.


In [33]:
function TwoQubitSystem(levels::Int)

    g_coupling = 0.1 # GHz (linear units)

    # annihilation operator for qubit 1 
    â = QuantumCollocation.lift(annihilate(levels), 1, 2)

    # annihilation operator for qubit 2
    b̂ = QuantumCollocation.lift(annihilate(levels), 2, 2);

    # drift Hamiltonian
    H_drift = 2π * g_coupling * â' * â * b̂' * b̂

    # drive Hamiltonians
    H_drives = [
        2π * (â + â'),
        2π * 1.0im * (â - â'),
        2π * (b̂ + b̂'),
        2π * 1.0im * (b̂ - b̂') 
    ]

    return QuantumSystem(H_drift, H_drives);
end;

In [34]:
levels_per_qubit = 2
system = TwoQubitSystem(levels_per_qubit);

## CNOT Unitary Gate

The goal gate that we will be optimizing for is the CNOT gate defined as

$$
U_{CNOT} = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{pmatrix}
$$


In [35]:
U_goal = [
    1 0 0 0;
    0 1 0 0;
    0 0 0 1;
    0 0 1 0
]; 

## Time Discretization Parameters

Here we will define the temporal discretization parameters for the control functions. We will use a total time of $T = 15$ and a time step of $\Delta t = 0.1$.

In [36]:
T = 15.0 # ns
N = 100
Δt = T / N;

## Pulse Constraints

Here we define bounds on the control as well as a constraint on the second derivative of the control that enforces smoothness of the pulse.

In [37]:
a_bound = 0.0175 # GHz
dda_bound = 0.2; # bound on second derivative of control, to ensure smoothness

## Defining The Problem

Here we define the problem using the `UnitarySmoothPulseProblem` which handles setting up a smoothness constrained pulse optimization problem for a unitary gate.

In [38]:
# setting maximum number of iterations 
max_iter = 100 

# TODO: change const violation tolerance
prob = UnitarySmoothPulseProblem(
    system,
    U_goal,
    N,
    Δt;
    a_bound=a_bound,
    dda_bound=dda_bound,
    max_iter=max_iter,
)

applying constraint: initial value of Ũ⃗
applying constraint: 

initial value of a
applying constraint: final value of a
applying constraint: bounds on a


applying constraint: bounds on dda
applying constraint: bounds on Δt
applying constraint: time step all equal constraint


QuantumControlProblem(Ipopt.Optimizer, MathOptInterface.VariableIndex[MOI.VariableIndex(1) MOI.VariableIndex(46) … MOI.VariableIndex(4411) MOI.VariableIndex(4456); MOI.VariableIndex(2) MOI.VariableIndex(47) … MOI.VariableIndex(4412) MOI.VariableIndex(4457); … ; MOI.VariableIndex(44) MOI.VariableIndex(89) … MOI.VariableIndex(4454) MOI.VariableIndex(4499); MOI.VariableIndex(45) MOI.VariableIndex(90) … MOI.VariableIndex(4455) MOI.VariableIndex(4500)], QuantumSystem{Float64}([0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.6283185307179586], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], [[0.0 0.0 6.283185307179586 0.0; 0.0 0.0 0.0 6.283185307179586; 6.283185307179586 0.0 0.0 0.0; 0.0 6.283185307179586 0.0 0.0], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; -0.0 0.0 0.0 0.0; 0.0 -0.0 0.0 0.0], [0.0 6.283185307179586 0.0 0.0; 6.283185307179586 0.0 0.0 0.0; 0.0 0.0 0.0 6.283185307179586; 0.0 0.0 6.283185307179586 0.0], [0.0 0.0 0.0 0.0; -0.0 0.0 0.0 0.0; 0.0 0.0 

Here we plot the initial guess for the unitary and controls, which are stored as `Symbol`s, `:Ũ⃗` and `:a`, respectively. Note that the initial guess for the unitary is not random, it is in fact the geodesic path between the identity and the target unitary. This is not required for the optimizer to converge, but it can help speed up convergence.

In [ ]:
plot(prob.trajectory, [:Ũ⃗, :a]; ignored_labels=[:Ũ⃗])

In [40]:
traj = prob.trajectory
traj.names

(:Ũ⃗, :a, :da, :dda, :Δt)

In [41]:
traj.dim * traj.T

4500

Here we call the `solve!` method on the problem, which uses the interior point nonlinear solver IPOPT to optimize over the states and controls of the problem.

In [42]:
QuantumCollocation.solve!(prob)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.0.

Number of nonzeros in equality constraint Jacobian...:    64774
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    34296

Total number of variables............................:     4460
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      892
                     variables with only upper bounds:        0
Total number of equality constraints.................:     4059
Total number of inequality c

   1  5.5096811e+01 4.16e-02 2.50e+01  -1.7 1.71e+00    -  1.00e+00 1.00e+00h  1


   2  1.4290071e+01 3.96e-02 2.61e+01  -1.7 3.00e+01    -  4.49e-02 4.73e-02f  1


   3  1.2876599e+01 3.60e-02 2.86e+01  -1.7 3.04e+00    -  4.48e-02 1.83e-01f  1


   4  4.3353872e+00 3.64e-02 2.74e+01  -1.7 3.31e+00   0.0 3.75e-02 4.92e-02f  1


...

  97  1.7970832e-03 1.63e-07 1.34e-04  -4.0 3.28e-03  -1.6 1.00e+00 1.00e+00H  1


  98  1.6236787e-03 5.68e-06 7.43e-04  -4.0 7.54e-03  -2.0 1.00e+00 1.00e+00H  1


  99  1.6075952e-03 5.07e-06 4.96e+01  -4.0 1.35e-02  -2.5 1.00e+00 1.25e-01h  4


iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100  1.8011890e-03 4.22e-06 1.24e+01  -4.0 1.73e-02  -3.0 1.00e+00 2.50e-01h  3



Number of Iterations....: 100

                                   (scaled)                 (unscaled)
Objective...............:   1.8011889877481355e-03    1.8011889877481355e-03
Dual infeasibility......:   1.2411278888233566e+01    1.2411278888233566e+01
Constraint violation....:   4.2245286799789646e-06    4.2245286799789646e-06
Variable bound violation:   0.0000000000000000e+00    0.0000000000000000e+00
Complementarity.........:   1.1765102673099178e-04    1.1765102673099178e-04
Overall NLP error.......:   1.2411278888233566e+01    1.2411278888233566e+01


Number of objective function evaluations             = 195
Number of objective gradient evaluations             = 101
Number of equality constraint evaluations            = 195
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 101
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 100
Total seconds in IPOPT  

Let's now calculate the fidelity of the optimized unitary with the target unitary, via rollout with the full matrix exponential, this is achieved via the `unitary_fidelity` method.

In [43]:
fid = unitary_fidelity(prob.trajectory, system)
println("Fidelity: $fid")

Fidelity: 0.9999951723559903


And now let's plot the final solution for the unitary and controls.

In [44]:
plot(prob.trajectory, [:Ũ⃗, :a]; ignored_labels=[:Ũ⃗])

UndefVarError: UndefVarError: `plot` not defined

## Minimum Time CNOT

Now let's see how fast we can make this pulse, by setting up and solving a minimum time problem, using the solution we just found.

We will add an objective term to the problem that penalizes the total time, as well as constraint on the final state of the form

$$
\mathcal{F}(U(T)) \geq \mathcal{F}_{\text{target}}.
$$

This enforce that the final fidelity does not decrease below the threshold $\mathcal{F}_{\text{target}}$, while also allowing the phase to shift.

In [45]:
# final fidelity constraint
min_fidelity = 0.9999

# minimum time objective weight
D = 1000.0

# define the problem
mintime_prob = UnitaryMinimumTimeProblem(prob; D=D, final_fidelity=min_fidelity);

In [ ]:
# solving the problem
QuantumCollocation.solve!(mintime_prob; max_iter=max_iter)

In [ ]:
mintime_final_fidelity = unitary_fidelity(mintime_prob.trajectory, system)
println("Fidelity: $mintime_final_fidelity")

In [ ]:
plot(mintime_prob.trajectory, [:Ũ⃗, :a]; ignored_labels=[:Ũ⃗])

In [ ]:
original_duration = times(prob.trajectory)[end]
mintime_duration = times(mintime_prob.trajectory)[end]
println("Original Duration: $original_duration ns")
println("Minimum Duration:  $mintime_duration ns")

## Model Mismatch and Iterative Learning Control

Now let's imagine that our goal is to test this pulse on an experimental system where we anticipate some hamiltonian errors. First we will define some states on this system using the `cavity_state` function to create states $\ket{g}$, $\ket{e}$ for individual qubits, and $\ket{gg}$, $\ket{ge}$, $\ket{eg}$, and $\ket{ee}$ on the combined two qubit system.

In [ ]:
experimental_levels = 2

# define quantum ground state |g⟩
g = cavity_state(0, experimental_levels)

# define quantum excited state |e⟩
e = cavity_state(1, experimental_levels)

# define quantum states |gg⟩, |ge⟩, |eg⟩, |ee⟩
gg = g ⊗ g
ge = g ⊗ e
eg = e ⊗ g
ee = e ⊗ e;

Here we set up the experiments for the system using the `QuantumSimulationExperiment` constructor.

In [ ]:
# unitary fidelity measurement function
unitary_fidelity_measurement = Ũ⃗ -> begin
    U_experiment = iso_vec_to_operator(Ũ⃗)
    return [unitary_fidelity(U_experiment, U_goal)]
end

experimental_initial_states = [
    gg,
    ge,
    eg,
    ee
]

measurement = Ũ⃗ -> begin
    U_experiment = iso_vec_to_operator(Ũ⃗)
    return [fidelity(U_experiment * ψ, U_goal * ψ) for ψ ∈ experimental_initial_states]
end


We will assume model mismatch of the form

$$
H_{\text{error}} = \epsilon_X (X \otimes I + I \otimes X) + \epsilon_Y (Y \otimes I + I \otimes Y) + \epsilon_Z (Z \otimes I + I \otimes Z)
$$

In [ ]:

X_error = 0.002 
Y_error = -0.007
Z_error = 0.006 

H_error = -1.0im * (
    X_error * (GATES[:X] ⊗ GATES[:I] + GATES[:I] ⊗ GATES[:X]) +
    Y_error * (GATES[:Y] ⊗ GATES[:I] + GATES[:I] ⊗ GATES[:Y]) +
    Z_error * (GATES[:Z] ⊗ GATES[:I] + GATES[:I] ⊗ GATES[:Z])
) 

G_error = iso(H_error)

In [ ]:



# define the measurement times
τs = [prob.trajectory.T]

# define initial unitary
U_init = I(4) 

# define the simulation experiment
experiment = QuantumSimulationExperiment(
    system,
    operator_to_iso_vec(U_init),
    measurement,
    τs;
    unitary=true,
    G_error_term=G_error
);

# define the simulation experiment
unitary_fidelity_experiment = QuantumSimulationExperiment(
    system,
    operator_to_iso_vec(U_init),
    unitary_fidelity_measurement,
    τs;
    unitary=true,
    G_error_term=G_error
);



Here we define the reference trajectory and measure the unitary fidelity on the target system

In [ ]:
# select goal trajectory
# -------------------------------------
# goal_prob = prob
goal_prob = mintime_prob

# calculate experimental fidelity
experiment_fidelity = unitary_fidelity_experiment(
    goal_prob.trajectory.a, 
    goal_prob.trajectory.Δt
).ys[end][end]

println("experimental fidelity = ", experiment_fidelity)

Here we define and solve the ILC problem using the `ILCProblem` constructor and `solve!` method.

In [ ]:
# backtrackign line search intial step size
α = 0.5^0

ILC_max_iter = 100

final_ILC_fidelities = []

Qfs = range(10.0, 1000.0; step=10.0)

for Qf ∈ Qfs 
    ILC_prob = ILCProblem(goal_prob, experiment; 
        α=α, 
        max_iter=ILC_max_iter,
        Qf=Qf,
        verbose=false,
        optimizer_verbose=false
    )

    solve!(ILC_prob; verbose=false);

    final_ILC_fidelity = unitary_fidelity_experiment(ILC_prob.Zs[end].a, ILC_prob.Zs[end].Δt).ys[end][end]

    push!(final_ILC_fidelities, final_ILC_fidelity)
end




In [ ]:
println(length(Qfs))
indices = 10:length(Qfs) 
lines(collect(Qfs[indices]), Float64[final_ILC_fidelities[indices]...])

In [ ]:
# backtrackign line search intial step size
α = 0.5^0

ILC_max_iter = 100

final_ILC_fidelities = []

Qf = 700.0

ILC_prob = ILCProblem(goal_prob, experiment; 
    α=α, 
    max_iter=ILC_max_iter,
    Qf=Qf,
)

solve!(ILC_prob);



In [ ]:
animation_path = "two_qubit.gif"
save_animation(ILC_prob, animation_path; plot_states=false, fps=5);

In [ ]:
final_ILC_fidelity = unitary_fidelity_experiment(ILC_prob.Zs[end].a, ILC_prob.Zs[end].Δt).ys[end][end]
initial_ILC_fidelity = unitary_fidelity_experiment(ILC_prob.Zs[1].a, ILC_prob.Zs[1].Δt).ys[end][end]
println("initial ILC fidelity: $initial_ILC_fidelity")
println("final ILC fidelity:   $final_ILC_fidelity")

In [ ]:
final_ILC_infidelity = 1.0 - final_ILC_fidelity
initial_ILC_infidelity = 1.0 - initial_ILC_fidelity
println("initial ILC infidelity: $initial_ILC_infidelity")
println("final ILC infidelity:   $final_ILC_infidelity")

In [ ]:
improvement_percentage = round((final_ILC_infidelity - initial_ILC_infidelity) / initial_ILC_infidelity * 100, digits=2)
println("ILC relative infidelity decrease: ", improvement_percentage, "%")

In [ ]:
flush(io)